In [19]:
import sys
sys.path.insert(0, '/home/cosmonaut/dev/pdp3/tools')
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
from scipy.fftpack import fft

from lib.parameters import Parameters

In [20]:
%matplotlib qt5

In [57]:
cfg = Parameters('../model32_single/parameters.xml')

In [58]:
def get_row_by_radius(radius):
    return(int(round(radius * cfg.r_grid_count / cfg.r_size)))

def get_col_by_longitude(longitude):
    return(int(round(longitude * cfg.z_grid_count / cfg.z_size)))

In [64]:
# configure here
#      ||
#      \/
types = ['r', 'z']
start_time = 0
# end_time = 7.902000e-09
end_time = 4.299000e-09
# end_time = 8.621000e-09
# end_time = 1.146000e-08
radius = 0.01
longitude = 0.2
#
#
#

In [65]:
step_interval = cfg.step_interval
suffix = "{}_{}_{}".format(cfg.data_dump_interval, get_row_by_radius(radius), get_col_by_longitude(longitude))

In [66]:
xf_s = []
yf_s = []

In [68]:
    for i in types:
        y = np.fromfile("%s/.cache/plot2d_%s_%e-%e-%e#%s" % (cfg.data_path, i, start_time, end_time, cfg.step_interval, suffix))
        N = len(y)
        sampling_rate = 1 / (cfg.step_interval * cfg.data_dump_interval)
        x = np.linspace(0, end_time, N)

        # Nyquist Sampling Criteria
        T = 1 / sampling_rate # inverse of the sampling rate
        x_f = np.linspace(0.0, 1.0/(2.0*T), int(N/2))
        xf_s.append(x_f)

        # FFT algorithm
        yr = fft(y) # "raw" FFT with both + and - frequencies
        if i == 'h_phi':
            y_f = 2/N * np.abs(yr[0:np.int(N/2)]) * 1000 # positive freqs only
        else:
            y_f = 2/N * np.abs(yr[0:np.int(N/2)]) # positive freqs only

        yf_s.append(y_f)
        plt.plot(x_f, y_f)

In [82]:
## two FFTs half time each
##
for i in range(0, len(types)):
    y = np.fromfile("%s/.cache/plot2d_%s_%e-%e-%e#%s" % (cfg.data_path, types[i], start_time, end_time, cfg.step_interval, suffix))
    N = len(y)
    N_half = int(N / 2)
    y_half = y[0:N_half]
    sampling_rate = 1 / (cfg.step_interval * cfg.data_dump_interval)
    x = np.linspace(0, end_time, N)
    
    x_half = x / 2
    
    # Nyquist Sampling Criteria
    T = 1 / sampling_rate # inverse of the sampling rate
    x_f = np.linspace(0.0, 1.0/(2.0*T), int(N/2))
    x_f_half = np.linspace(0.0, 1.0/(2.0*T), int(N_half/2))
    
    # FFT algorithm
    yr = fft(y) # "raw" FFT with both + and - frequencies
    yr_half = fft(y_half) # "raw" FFT with both + and - frequencies
    
    y_f = 2/N * np.abs(yr[0:np.int(N/2)]) # positive freqs only
    y_f_half = 2/N_half * np.abs(yr_half[0:np.int(N_half/2)]) # positive freqs only
    plt.plot(x_f, y_f, x_f_half, y_f_half)